In [176]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [177]:
import torch
from copy import deepcopy
from PFNExperiments.Evaluation.CompareModelToGT import CompareModelToGT, convert_to_batchsize_1_dataloader

In [178]:
from PFNExperiments.LinearRegression.GenerativeModels.GenerateData import GenerateData, check_data, check_and_plot_data
from PFNExperiments.LinearRegression.GenerativeModels.GenerateDataLM_Examples import make_lm_program_plain, make_lm_program_gamma_gamma, make_lm_program_gamma_gamma_augmented, make_lm_program_gamma_gamma_batched, make_make_lm_program_gamma_gamma_batched_quantized
from PFNExperiments.LinearRegression.GenerativeModels.GenerateX import simulate_X_uniform
from PFNExperiments.LinearRegression.GenerativeModels.LM_abstract import return_only_y, print_code
from PFNExperiments.LinearRegression.GenerativeModels.Quantizer import Quantizer
from PFNExperiments.LinearRegression.Models.Transformer import Transformer
from PFNExperiments.Training.Trainer import Trainer, batch_to_model_lm, visualize_training_results
from PFNExperiments.Training.Losses import MSELoss_unsqueezed, nll_loss_full_gaussian
from PFNExperiments.Training.EvalMetrics import mean_squared_error_torch_avg, mae_torch_avg, r2_score_torch_avg
from PFNExperiments.LinearRegression.Models.ModelPosterior import ModelPosteriorFullGaussian, ModelPosteriorFullGaussian2
from PFNExperiments.LinearRegression.Models.AugmentLoss import add_l2_loss_nll_loss

from PFNExperiments.Evaluation.ComparePosteriorSamples import compare_all_metrics, marginal_plots_hist_parallel, marginal_plots_kde_together
from PFNExperiments.LinearRegression.Evaluation.CompareModels import ModelComparison
from PFNExperiments.LinearRegression.Models.ModelToPosterior import ModelToPosterior
from PFNExperiments.LinearRegression.ComparisonModels.Hamiltionion_MC import Hamiltionian_MC

from PFNExperiments.LinearRegression.ComparisonModels.Variational_Inference import Variational_Inference, make_guide_program_gamma_gamma

from PFNExperiments.LinearRegression.Evaluation.CompareComparisonModels import CompareComparisonModels

from PFNExperiments.LinearRegression.GenerativeModels.Curriculum import Curriculum
from PFNExperiments.LinearRegression.GenerativeModels.GenerateDataCurriculum import GenerateDataCurriculum

from PFNExperiments.Training.TrainerCurriculum import TrainerCurriculum

from PFNExperiments.LinearRegression.ComparisonModels.AnalyticalSolutionsLM import PosteriorLM_IG
from PFNExperiments.LinearRegression.GenerativeModels.GenerateDataLM_Examples import make_lm_program_ig_batched, make_lm_program_ig
from PFNExperiments.LinearRegression.GenerativeModels.GenerateX import simulate_X_uniform

from PFNExperiments.LinearRegression.Evaluation.CompareComparisonModels import CompareComparisonModels

In [179]:
N = 100
P = 10

N_EPOCHS = 100
N_BATCHES_PER_EPOCH = 10
BATCH_SIZE = 256
N_SAMPLES_PER_EPOCH = N_BATCHES_PER_EPOCH * BATCH_SIZE

In [180]:
a = 5.0
b = 2.0
tau = 1.0
curriculum = Curriculum(max_iter=int(N_EPOCHS*N_BATCHES_PER_EPOCH*BATCH_SIZE*0.5))
curriculum.add_param_list(
    [
    ("tau", curriculum.constant_scheduler(tau)),
    ("a", curriculum.constant_scheduler(a)),
    ("b", curriculum.constant_scheduler(b)),

    ]
)
#curriculum.plot_all_schedules()
curriculum.get_params(299*N_SAMPLES_PER_EPOCH)
data_generator = GenerateDataCurriculum(
    pprogram_maker= make_lm_program_ig_batched,
    curriculum= curriculum,
)

In [181]:
data_generator = GenerateDataCurriculum(
    pprogram_maker= make_lm_program_ig_batched,
    curriculum= curriculum,
)

In [182]:
epoch_loader = data_generator.make_epoch_loader(
    n = N,
    p = P,
    number_of_batches_per_epoch = N_BATCHES_PER_EPOCH,
    n_epochs = N_EPOCHS,
    batch_size= BATCH_SIZE,
    train_frac= 0.5,
    val_frac= 0.1,
    shuffle=False,
    n_samples_to_generate_at_once = 250_000
)

The number of batches times the number of epochs must be equal to the total number of iterations in the curriculum. But got 256000 and 128000 respectively


In [183]:
train_loader = epoch_loader[0][0]

evaluation_loader = convert_to_batchsize_1_dataloader(train_loader)

In [184]:
test_case = next(iter(evaluation_loader))

In [204]:
gt_samples = []
for i in range(10):
    gt_samples.append(next(iter(evaluation_loader)))

gt_samples2 = []
for i in range(10):
    gt_samples2.append(next(iter(evaluation_loader)))

In [205]:
def fake_posterior_samples_randn(test_case, n_samples = 100):
    new_test_case = deepcopy(test_case)
    new_test_case["beta"] = torch.randn(n_samples, *test_case["beta"].shape[1:])
    return new_test_case

def fake_posterior_samples_multivariate_normal(test_case, n_samples = 100):
    new_test_case = deepcopy(test_case)
    new_test_case["beta"] = torch.distributions.MultivariateNormal(test_case["beta"], 1e-3*torch.eye(test_case["beta"].shape[1])).sample((n_samples,)).squeeze(1)
    return new_test_case

In [206]:
model_posterior_samples1 = []
for test_case in gt_samples:
    model_posterior_samples1.append(fake_posterior_samples_multivariate_normal(test_case, n_samples = 1000))

In [207]:
model_posterior_samples1[0]["beta"].shape 

torch.Size([1000, 10])

In [211]:
compare_model_to_gt = CompareModelToGT(n_draws_posterior_to_sample_joint=3)

In [229]:
r = compare_model_to_gt.compare_gt_parameters(
    ground_truth_data=gt_samples,
    model_samples=model_posterior_samples1,
)

  0%|          | 0/10 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [172]:
r

[{'Perplextiy': array([-21.87390467]),
  'MAP diff': tensor(0.7112, dtype=torch.float64),
  'Mean diff': tensor(0.0918)},
 {'Perplextiy': array([-22.557175]),
  'MAP diff': tensor(0.9602, dtype=torch.float64),
  'Mean diff': tensor(0.0645)},
 {'Perplextiy': array([-22.00819064]),
  'MAP diff': tensor(1.1366, dtype=torch.float64),
  'Mean diff': tensor(0.1077)},
 {'Perplextiy': array([-22.4637524]),
  'MAP diff': tensor(1.0646, dtype=torch.float64),
  'Mean diff': tensor(0.0907)},
 {'Perplextiy': array([-22.19601496]),
  'MAP diff': tensor(0.7827, dtype=torch.float64),
  'Mean diff': tensor(0.0710)},
 {'Perplextiy': array([-21.81990033]),
  'MAP diff': tensor(0.8617, dtype=torch.float64),
  'Mean diff': tensor(0.0788)},
 {'Perplextiy': array([-22.10663265]),
  'MAP diff': tensor(0.6948, dtype=torch.float64),
  'Mean diff': tensor(0.0732)},
 {'Perplextiy': array([-22.23147748]),
  'MAP diff': tensor(0.9064, dtype=torch.float64),
  'Mean diff': tensor(0.0914)},
 {'Perplextiy': array([-22.

In [231]:
r2 = compare_model_to_gt.compare_joint_samples(
    ground_truth_data=gt_samples2,
    model_samples=model_posterior_samples1,
)

  0%|          | 0/3 [00:00<?, ?it/s]

torch.Size([10, 1110]) torch.Size([10, 1110])
torch.Size([10, 1110]) torch.Size([10, 1110])
torch.Size([10, 1110]) torch.Size([10, 1110])


 33%|███▎      | 1/3 [00:04<00:08,  4.46s/it]

torch.Size([10, 1110]) torch.Size([10, 1110])
torch.Size([10, 1110]) torch.Size([10, 1110])
torch.Size([10, 1110]) torch.Size([10, 1110])


 67%|██████▋   | 2/3 [00:07<00:03,  3.73s/it]

torch.Size([10, 1110]) torch.Size([10, 1110])
torch.Size([10, 1110]) torch.Size([10, 1110])
torch.Size([10, 1110]) torch.Size([10, 1110])


100%|██████████| 3/3 [00:11<00:00,  3.87s/it]


In [232]:
r2

[{'Wasserstein_distance with metric euclidean': 23.1306209564209,
  'MMD': 0.5882294178009033,
  'CST_accuracy': 0.44999998807907104,
  'CST_roc_auc_score': 0.45},
 {'Wasserstein_distance with metric euclidean': 23.13232421875,
  'MMD': 0.5880863666534424,
  'CST_accuracy': 0.6499999761581421,
  'CST_roc_auc_score': 0.65},
 {'Wasserstein_distance with metric euclidean': 23.131898880004883,
  'MMD': 0.5882654190063477,
  'CST_accuracy': 0.699999988079071,
  'CST_roc_auc_score': 0.7}]